<a href="https://colab.research.google.com/github/danielbauer1979/MSDIA_PredictiveModelingAndMachineLearning/blob/main/GB886_VII_3_DeTrending.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Univariate Time Series

This codebook goes through a few time series examples to showcase basics of univariate time series modeling for generating forecasts. We first look at some basic examples where we carry the steps out manually. We then show how packages and particularly Meta's Prophet can automate the analysis.


As usually, let's load relevant libraries where we rely on time series functionality in statsmodels for our analysis

In [ ]:
# Standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Time series functionality
from statsmodels.tsa.stattools import adfuller #ADF test for stationarity
from statsmodels.tsa.seasonal import seasonal_decompose

## Two Examples

We will go through two introductory examples to demonstrate different types of time series: We look at the S&P500 stock index (daily observations) as one example, and an example of retail sales (monthly observations) as a second example.

### Retail Sales Data

Let's load the data:

In [ ]:
!git clone https://github.com/danielbauer1979/MSDIA_PredictiveModelingAndMachineLearning.git

In [ ]:
dat_sales = pd.read_csv('MSDIA_PredictiveModelingAndMachineLearning/GB886_VII_3_RetailSales.csv')
dat_sales.head()

The data goes from January 1992 (index zero) to May 2016 (last index). Let's plot it:

In [ ]:
plt.plot(dat_sales)
plt.title('Retail Sales')
plt.show()

It is from **visual inspection** [link text](https://)evident that there is seasonality---the same pattern seems to occur over and over again. This is no suprise for retail sales (increased sales around the holidays, say), so our intuition is also helpful. Furthermore, there seems to be a time trend (the series trends upwards).

So, to remove the trend, we use the "decomposition" functionality in statsmodels:

In [ ]:
# Perform seasonal decomposition
result = seasonal_decompose(dat_sales, model='additive', period=12)

# Plot the trend component
plt.plot(result.trend)
plt.title('Trend Component of Retail Sales')
plt.show()

#Plot the seasonal component
plt.plot(result.seasonal)
plt.title('Seasonal Component of Retail Sales')
plt.show()

# Plot the residual component
plt.plot(result.resid)
plt.title('Residual Component of Retail Sales')
plt.show()

We notice the clear trend and the seasonal pattern (which keeps on repeating). Let's check whether the residual is stationary using the *Augmented Dickey Fuller (ADF) test*, where the Null hypothesis is *non-stationarity*---so, if the p-value is small and we reject, there is evidence for stationarity:

In [ ]:
result = adfuller(result.resid.dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))


So, it appears that the ADF test rejects, so it seems after removing trend and seasonality, the data appears stationary!

### S&P 500 Daily Data

Let's now load and look at the stock data:

In [ ]:
dat_sandp = pd.read_csv('MSDIA_PredictiveModelingAndMachineLearning/GB886_VII_3_SanP500.csv')
dat_sandp.head()

The data is from 2/28/2023 (index zero) until 2/28/2024 (last index). Let's take a quick look:

In [ ]:
plt.plot(dat_sandp)
plt.title('Evolution of S and P Index 3/2023 to 3/2024')
plt.show()

So, the first question is to check whether the time series is stationary. Here, **visual inspection** again makes it clear that the data appears not stationary, although there is also not a very clear deterministic trend.

Let's check the ADF test:

In [ ]:
result = adfuller(dat_sandp)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))


So, here the ADF test does not reject so we **do not** have stationarity. So, let's take the **first differences**, i.e., let's look at
$$
y^d_t = y_t - y_{t-1}
$$
and then work with the differenced series $\{y^d_1,y^d_2,y^d_3,...\}$:

In [ ]:
dat_sandp_diff = dat_sandp.diff().dropna()
plt.plot(dat_sandp_diff)
plt.title('First Differences of S and P Index 3/2023 to 3/2024')
plt.show()

So, there is no longer an indication of a trend. Let's run the ADF test on the differenced series.

In [ ]:
result = adfuller(dat_sandp_diff)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

So, that looks stationary. So, we can continue with our anaylsis using the differenced series.